In [1]:
import sys
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import MDAnalysis as mda
import MDAnalysis.transformations as trans
import nglview as nv


import warnings
warnings.filterwarnings(action='once')


if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *


/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


/home/anna/Documents/cinnabar/cinnabar/__init__.py
adding code to the pythonpath...


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [2]:
prot_file = "/home/anna/Documents/benchmark/tyk2_benchmark/execution_model/protocol_2fs.dat"
protocol = pipeline_protocol(prot_file)
protocol.validate()
protocol.fepprep("middle")

protocol.num_lambda(11)
protocol.engine("AMBER")

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f8dbceb59a0>
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp3qpj4566'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpwjhv1tgg'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp_umnojxc'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpcyt2vhyv'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <T

hmr factor not found in protocol. auto will be used.
engines not found in protocol. ALL will be used.
fepprep not found in protocol. middle will be used.
'hmr' must be set to True for a hmr factor to be applied. It will still be set as auto.


'AMBER'

In [3]:
# Load equilibrated inputs for both ligands
prep_dir = "/home/anna/Documents/benchmark/tyk2_benchmark/prep"
lig_1 = "lig_ejm31"
lig_2 = "lig_ejm48"

for name, leg in zip(["lig", "sys"], ["free", "bound"]):
    # Load equilibrated inputs for both ligands
    system_1 = BSS.IO.readMolecules(
        [f"{prep_dir}/{lig_1}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_1}_{name}_equil_solv.prm7"])
    system_2 = BSS.IO.readMolecules(
        [f"{prep_dir}/{lig_2}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_2}_{name}_equil_solv.prm7"])

    print(f"Preparing the {leg} leg...")
    if leg == "free":
        system_free = merge.merge_system(system_1, system_2)
    if leg == "bound":
        system_bound = merge.merge_system(system_1, system_2)


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.

Preparing the free leg...
mapping, aligning and merging the ligands...
Preparing the bound leg...
mapping, aligning and merging the ligands...


In [4]:
pmemd_path = os.environ["AMBERHOME"] + "/bin/pmemd.cuda"

In [ ]:
fepprep_obj = fepprep(system_free, system_bound, protocol)
free_system, bound_system = fepprep_obj.prep_system_middle(pmemd_path)
# fepprep_obj.generate_folders("/home/anna/Documents/code/test/lig_ejm31~lig_ejm48")

In [5]:
lig_fep = "fepprep"
engine="AMBER"

protocol_min_rest, protocol_min = min_prots(lig_fep)

protocol_nvt_sol, protocol_nvt_heavy, protocol_nvt = nvt_prots(lig_fep)

protocol_npt_heavy, protocol_npt_heavy_lighter, protocol_npt = npt_prots(lig_fep)

In [27]:
minimised1 = run_process(system_bound, protocol_min_rest, engine, pmemd_path)
minimised2 = run_process(minimised1, protocol_min, engine, pmemd_path)

In [ ]:
equil1 = run_process(minimised2, protocol_nvt_sol, engine, pmemd_path)


In [ ]:
equil2 = run_process(equil1, protocol_nvt_heavy, engine, pmemd_path)


In [ ]:
equil3 = run_process(equil2, protocol_nvt, engine, pmemd_path)

In [24]:
BSS.Notebook.View(equil2).system()

NGLWidget(gui_style='ngl')

In [20]:
equil4 = run_process(equil3, eq_protocol, engine, pmemd_path, work_dir="eq_test")

In [26]:
BSS.Notebook.View(equil4).system()

NGLWidget(gui_style='ngl')

In [ ]:
equil4 = run_process(equil3, protocol_npt_heavy, engine, pmemd_path, work_dir="eq_test2")


In [ ]:
equil5 = run_process(equil4, protocol_npt_heavy_lighter, engine, pmemd_path)

In [ ]:
sys_equil_fin = run_process(equil5, protocol_npt, engine, pmemd_path)

In [ ]:
fepprep_obj = fepprep(system_free, system_bound, protocol)
free_system, bound_system = fepprep_obj.prep_system_middle(pmemd_path)
# fepprep_obj.generate_folders("/home/anna/Documents/code/test/lig_ejm31~lig_ejm48")

In [15]:
args = {"lam":0.5, "lam_vals":[0.5]}

min_protocol = BSS.Protocol.FreeEnergyMinimisation(
                                                num_lam=protocol.num_lambda(),
                                                steps=protocol.min_steps(), **args
                                                )
heat_protocol = BSS.Protocol.FreeEnergyEquilibration(timestep=protocol.timestep()*protocol.timestep_unit(),
                                                    num_lam=protocol.num_lambda(),
                                                    runtime=protocol.eq_runtime()*protocol.eq_runtime_unit(),
                                                    pressure=None,
                                                    temperature_start=protocol.start_temperature()*protocol.temperature_unit(),
                                                    temperature_end=protocol.end_temperature()*protocol.temperature_unit(),
                                                    hmr_factor=protocol.hmr_factor(), **args
                                                    )
eq_protocol = BSS.Protocol.FreeEnergyEquilibration(timestep=protocol.timestep()*protocol.timestep_unit(),
                                                num_lam=protocol.num_lambda(),
                                                runtime=protocol.eq_runtime()*protocol.eq_runtime_unit(),
                                                temperature=protocol.temperature()*protocol.temperature_unit(),
                                                pressure=protocol.pressure()*protocol.pressure_unit(),
                                                restart=True,
                                                hmr_factor=protocol.hmr_factor(), **args
                                                )
freenrg_protocol = BSS.Protocol.FreeEnergy(timestep=protocol.timestep()*protocol.timestep_unit(),
                                            num_lam=protocol.num_lambda(),
                                            runtime=protocol.sampling()*protocol.sampling_unit(),
                                            temperature=protocol.temperature()*protocol.temperature_unit(),
                                            pressure=protocol.pressure()*protocol.pressure_unit(),
                                            restart=True,
                                            hmr_factor=protocol.hmr_factor(), **args
                                        )

In [7]:
minimised1 = run_process(system_free, min_protocol, "AMBER", pmemd_path)

In [8]:
eq_sys = run_process(minimised1, heat_protocol, "AMBER", pmemd_path)

In [9]:
eq2_sys = run_process(eq_sys, eq_protocol, "AMBER", pmemd_path)

In [26]:
BSS.Notebook.View(eq_sys).system()

NGLWidget(gui_style='ngl')

In [ ]:
final_sys = run_process(eq2_sys, freenrg_protocol, "AMBER", pmemd_path)

In [ ]:
free_system.nPerturbableMolecules()

In [ ]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm44~lig_ejm45","lig_ejm44~lig_ejm49"]
engine = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["2fs"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        # try:
        # using the pipeline module for analysis
        analysed_pert = analyse(path_to_dir)
        analysed_pert.set_options(analysis_options)
        analysed_pert.set_options({"try pickle":False})
        avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # analysed_pert.plot_graphs()
        write_analysis_file(analysed_pert, final_results_folder, method=method)
        # except Exception as e:
        #     print(e)
        #     print(f"could not analyse {path_to_dir}")

In [ ]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm31~lig_ejm48","lig_ejm43~lig_ejm46", "lig_ejm46~lig_jmc28", "lig_jmc27~lig_jmc28"]
engine = "AMBER"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["both_failed_wrong_lambda_windows_used"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        # trans = f"{trans.split('~')[1]}~{trans.split('~')[0]}" # for reverse
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")